# Project 07 - Semaphores and Deadlocks

## Abstract

This week, we focus on the concept of a sempahore as a way to synchronize various processes in an efficent way all the while avoiding problems that come from multithreading (such as race conditions). We will implement a simple semaphore and test it against a producer/consumer program created by professor Al Madi. We will then use our semaphores to solve the dining philosopher's problem. 

## Results and Discussion

We build our semaphore by implementing an acquire and a release method. The acquire method sleeps a thread if the counter falls below zero and the release method wakes up a thread once the condition variable has been released. We test our function below: We have a place and a remove method implemented by professor Al Madi. The producer and the consumer methods below then place and remove items from a buffer. We expect to see no errors, the buffer size should never exceed ten, and all items should be removed from the buffer at the end of each run. 

In [1]:
import threading
import time
import random
import semaphore

class Buffer:
     
    def __init__(self, size):
        self.size = size
        self.queue = []
        self.lock = threading.Lock()
        self.full = semaphore.Semaphore(0)
        self.empty = semaphore.Semaphore(self.size)
        
    def place(self, item):
        self.empty.acquire()
        
        self.lock.acquire()
        self.queue.append(item)
        self.lock.release()
        
        self.full.release()

        
    def remove(self):
        self.full.acquire()
        
        self.lock.acquire()
        item = self.queue.pop(0)
        self.lock.release()
        
        self.empty.release()
        
        return item

In [2]:
my_buffer = Buffer(10)

In [3]:
def producer():
    for item in range(100):
        print("adding item:", item, "\tbuffer length:", len(my_buffer.queue))
        my_buffer.place(item)

In [4]:
def consumer():
    for index in range(100):
        item = my_buffer.remove()
        print("removing item:", item, "\tbuffer length:", len(my_buffer.queue))

In [5]:
t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)

t1.start()
t2.start()

t1.join()
t2.join()

adding item: 0 	buffer length: 0
adding item: 1 	buffer length: 1
adding item: 2 	buffer length:removing item: 0 	buffer length: 1
removing item: 1 	buffer length: 0
 2
adding item: 3 	buffer length: removing item: 2 	buffer length: 0
1
adding item: removing item: 3 	buffer length: 0
4 	buffer length: 1
adding item: 5 	buffer length: 1
removing item: 4 	buffer length: 0
adding item: 6 	buffer length: 1removing item: 5 	buffer length: 0

adding item: 7 	buffer length: 1
adding item: 8 removing item: 6 	buffer length: 1
removing item: 7 	buffer length: 0
	buffer length: 2
adding item:removing item: 8 	buffer length: 0
 9 	buffer length: 1
adding item: 10 	buffer length: 1
adding item: 11 	buffer length: removing item: 9 	buffer length: 1
removing item: 10 	buffer length: 0
2
adding item: 12 	buffer length: 1
adding item: 13 removing item: 11 	buffer length: 2
adding item: 14 	buffer length: 2
adding item: 15 	buffer length: 3
adding item: 16 	buffer length: 4
adding item: 17 	buffer leng

As can be seen in the run above, we never have the buffer length exceeding ten. Furthermore, we end with a buffer of length ten just as we wanted to. If we rerun the process above, we end up with the same results. This only strengthens the validty of our semaphore. Let's now try and solve the dining philosopher's problem. 

The dining problem is well known for generating a deadlock when we have two or more dining philosophers, who sit and eat, but for some reason need two forks while they eat. The deadlock arises when each of the philosophers picks up a fork, but can't pick up the second fork because it's already been taken by another philosopher. I implemented the dining philosopher's dilemma before. It can be done with any number of philosopher's greater than 1, and the default is 5. The below results in a deadlock. 

In [8]:
import dining_philosophers

dp = dining_philosophers.DiningPhilosophers()
dp.create_threads()

P 0  is thinking...
P 1  is thinking...
P 2  is thinking...
P 3  is thinking...
P 4  is thinking...
P 3  is taking left fork...
P 2  is taking left fork...
P 0  is taking left fork...
P 1  is taking left fork...
P 4  is taking left fork...
P 3  is taking right fork...
P 2  is taking right fork...
P 0  is taking right fork...
P 1  is taking right fork...
P 4  is taking right fork...


KeyboardInterrupt: 

P0-P4 take the left fork and then P0-P4 can't pick up the right fork since it's already taken. The print statements happen before the forks are picked up hence the illusion that the right fork has been picked up by all philosopher's but as can be seen, all threads result in a deadlock. 

Below, we implement the naive solution, the odd numbered philosphers pick up the left fork followed by the right fork and the even numbered philosophers pick up the right fork followed by the left fork. If we imagine each of the forks as semaphores, what should happen is that one of the philophers will be able to eat and thus finish eating. Once the forks are put down, we can expect another philosopher to be able to eat. Thus, this avoids deadlock. 

In [7]:
import dining_philosophers
dp = dining_philosophers.DiningPhilosophers(5)
dp.create_threads(solution = True)

P 0  is thinking...
P 1  is thinking...
P 2  is thinking...
P 3  is thinking...
P 4  is thinking...
P 2  is taking right fork...
P 0  is taking right fork...
P 3  is taking left fork...
P 1  is taking left fork...
P 4  is taking right fork...
P 2  is taking left fork...
P 0  is taking left fork...
P 3  is taking right fork...
P 2  is eating...
P 0  is eating...
P 2  dropped right fork...
P 2  dropped left fork...
P 2  is thinking...
P 2  is taking right fork...
PP 3  is eating...
 1  is taking right fork...
P 0  dropped right fork...
P 0  dropped left fork...
P 0  is thinking...
P 0  is taking right fork...
P 3  dropped right fork...
P 3  dropped left fork...
P 3  is thinking...
P 1  is eating...
P 3  is taking left fork...
P 1  dropped right fork...
P 1  dropped left fork...
P 1  is thinking...
P 0  is taking left fork...
P 4  is taking left fork...
P 1  is taking left fork...
P 2  is taking left fork...
P 0  is eating...
P 0  dropped right fork...
P 0  dropped left fork...
P 0  is th

The above shows all philosophers being able to finish. 

## Acknowledgements

I want to acknowledge Professor Al Madi for helping me debug my semaphore and for helping me debug the dining philosopher's program. Katherine Andre for emotional support.